 # **Semivariances → MHAR‑ReCov Spillovers (FULL PIPELINE)**



 End‑to‑end, reproducible code that:



 1. Loads intraday day‑ahead electricity prices *(Europe / Australia)*

 2. Computes hourly **simple returns**

 3. Aggregates to daily **positive** and **negative** realised semicovariance matrices (ReCov⁺, ReCov⁻)

 4. Converts each matrix to its *vech* vector

 5. Applies a **Probability‑Integral Transform (PIT)** element‑wise to stabilise variances

 6. Saves intermediary Parquet artefacts

 7. Runs two MHAR‑ReCov LASSO spillover analyses – one for ReCov⁺, one for ReCov⁻ –

    and prints previews (`head()`) plus Total Spillover Indices.



 Every major step prints a small preview so you can inspect the data as it flows

 through the pipeline.



 ---

 **How to run**

 * Save this file as `semivariances_pipeline.py` **or** open it directly in

   Jupyter/VS Code (each `# %%` marker is a cell).

 * Execute the notebook top‑to‑bottom (or click the green **Run pipeline**

   button after selecting *europe* / *australia*).

 * All outputs appear inline – scroll to watch the transformation.

 ---

In [17]:
# Imports & globals
import os, json, random, warnings
import numpy as np, pandas as pd
from scipy.stats import norm, rankdata
from tqdm.notebook import tqdm

# ML / stats
from sklearn.model_selection import KFold
from sklearn.linear_model import MultiTaskLassoCV

# Display helpers (Jupyter)
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

warnings.filterwarnings('ignore')
SEED = 12345
np.random.seed(SEED); random.seed(SEED)

# Helper shared by every MHAR model
def create_mhar_lags(df: pd.DataFrame) -> pd.DataFrame:
    lag1 = df.shift(1).add_suffix('_lag1')                       # daily
    wavg = df.rolling(7 , min_periods=7).mean().shift(1)         # weekly
    mavg = df.rolling(30, min_periods=30).mean().shift(1)        # monthly
    return pd.concat([df, lag1, wavg.add_suffix('_wavg'),
                             mavg.add_suffix('_mavg')], axis=1).dropna()


 ## 0 · Select study area

In [18]:
area_dd = widgets.Dropdown(options=['europe', 'australia'], value='europe',
                           description='Dataset:')
run_btn = widgets.Button(description='Run pipeline', button_style='success', icon='play')

# Display the UI controls
display(area_dd, run_btn)


Dropdown(description='Dataset:', options=('europe', 'australia'), value='europe')

Button(button_style='success', description='Run pipeline', icon='play', style=ButtonStyle())

 ## 1 · Load intraday prices → simple returns

In [19]:
PRICES_PATHS = {
    'europe':    'parquet_files/filtered_data.parquet',
    'australia': 'parquet_files/filtered_data_australia.parquet',
}

def load_prices(area: str) -> pd.DataFrame:
    """Return wide price DataFrame (index = timestamp, columns = areas)."""
    df = (pd.read_parquet(PRICES_PATHS[area])
            .sort_values(['Area', 'Start DateTime']))
    wide = (df.pivot(index='Start DateTime', columns='Area',
                     values='Day-ahead Price (EUR/MWh)')
              .sort_index())
    return wide

def simple_returns(prices: pd.DataFrame) -> pd.DataFrame:
    ret = np.log(prices).diff().dropna()         # log-returns
    ret['Date'] = ret.index.date
    return ret


def show(title, obj, n=5):
    """Pretty‑print a title + DataFrame/Series preview."""
    display(Markdown(f"### {title}"))
    if isinstance(obj, dict):          # show first item when dict of DFs
        k, v = next(iter(obj.items()))
        display(Markdown(f"*First key:* **{k}**"))
        display(v.head(n))
    else:
        display(obj.head(n))


 ## 2 · Daily realised semicovariances (ReCov⁺ / ReCov⁻)

In [20]:
def daily_semicov(ret: pd.DataFrame):
    """
    Return two dicts {date: DataFrame}: ReCov⁺ and ReCov⁻
    (daily realised positive / negative semicovariances).
    """
    pos, neg = {}, {}
    for day, grp in tqdm(ret.groupby('Date'), desc='daily semicov'):
        r = grp.drop(columns='Date')          # intraday returns (T × N)
        T, N = r.shape                        # T = intraday obs. per day
        cov_p = np.zeros((N, N))
        cov_n = np.zeros_like(cov_p)

        for row in r.values:
            rp = np.clip(row, 0, None)        # r⁺  (>= 0)
            rn = np.clip(row, None, 0)        # r⁻  (<= 0)
            cov_p += np.outer(rp, rp)
            cov_n += np.outer(rn, rn)

        # --- scale by intraday count ---------------------------------
        cov_p /= T
        cov_n /= T

        cols = r.columns
        pos[day] = pd.DataFrame(cov_p, index=cols, columns=cols)
        neg[day] = pd.DataFrame(cov_n, index=cols, columns=cols)

    return pos, neg



 ## 3 · vech vectorisation + PIT transform

In [21]:

def build_vech_dataframe(cov_dict):
    example = next(iter(cov_dict.values()))
    areas   = example.columns.tolist()
    labels = [f"{areas[i]}" if i == j else f"{areas[i]}-{areas[j]}"
              for i in range(len(areas))
              for j in range(i, len(areas))]   

    records, dates = [], []
    for day in sorted(cov_dict.keys()):
        mat = cov_dict[day].values
        records.append(mat[np.tril_indices(len(mat))])
        dates.append(day)
    df = pd.DataFrame(records, index=pd.to_datetime(dates), columns=labels)
    return df, labels

def pit_transform(df: pd.DataFrame):
    out = df.copy()
    for col in df.columns:
        x = df[col].values
        u = rankdata(x, method='average') / (len(x) + 1)
        u = np.clip(u, 1e-6, 1-1e-6)
        out[col] = norm.ppf(u)
    return out


 ## 4 · Helper: save long‑format semicovariances (optional)

In [22]:
PARQUET_DIR = 'parquet_files'
os.makedirs(PARQUET_DIR, exist_ok=True)

def flat_save(cov_dict, tag, area):
    rows = []
    for d, mat in cov_dict.items():
        for i, a in enumerate(mat.index):
            for j, b in enumerate(mat.columns):
                rows.append({'Date': d, 'Market1': a, 'Market2': b,
                             'Value': mat.iloc[i, j]})
    pd.DataFrame(rows).to_parquet(f"{PARQUET_DIR}/daily_semicov_{tag}_{area}.parquet")


 ## 5 · MHAR‑ReCov spillover function (static, H = 1)

In [23]:

def mhar_spillover(pit_path: str, area: str):
    """Estimate MHAR‑ReCov LASSO on PIT‑transformed *pit_path*.
    Returns spillover table and Total Spillover Index."""
    pit_vech = pd.read_parquet(pit_path)
    with open(f"{PARQUET_DIR}/vech_labels_{area}.json") as f:
        labels = json.load(f)
    pit_vech.columns = labels

    # --- MHAR lag construction ------------------------------------
    def create_mhar_lags(df):
        lag1 = df.shift(1).add_suffix('_lag1')
        wavg = df.rolling(7, min_periods=7).mean().shift(1).add_suffix('_wavg')
        mavg = df.rolling(30, min_periods=30).mean().shift(1).add_suffix('_mavg')
        return pd.concat([df, lag1, wavg, mavg], axis=1).dropna()

    mhar_data = create_mhar_lags(pit_vech)
    X = mhar_data.drop(columns=pit_vech.columns)
    Y = mhar_data[pit_vech.columns]

    # --- MultiTask LASSO -----------------------------------------
    cv = KFold(n_splits=5, shuffle=True, random_state=SEED)
    mtl = MultiTaskLassoCV(cv=cv, random_state=SEED, tol=1e-10,
                           max_iter=1_000_000).fit(X, Y)
    B = mtl.coef_
    ints = mtl.intercept_

    # --- Φ₁ (H=1) -------------------------------------------------
    Bd, Bw, Bm = np.split(B, 3, axis=1)
    Phi1 = Bd + Bw/7 + Bm/30

    # --- Residual Σ ----------------------------------------------
    Y_hat = X.values @ B.T + ints
    resid = Y.values - Y_hat
    Sigma = resid.T @ resid / resid.shape[0]

    # --- GVD ------------------------------------------------------
    K = Y.shape[1]
    # add A(0)=I and A(1)=Φ1
    A_list = [np.eye(K), Phi1]

    theta  = np.zeros((K, K))
    for i in range(K):
        e_i   = np.eye(K)[i]
        denom = sum(e_i @ A @ Sigma @ A.T @ e_i for A in A_list)
        for j in range(K):
            e_j   = np.eye(K)[j]
            numer = sum((e_i @ A @ Sigma @ e_j)**2 for A in A_list)
            theta[i, j] = inv_diag[j, j] * numer / denom
    # normalise rows – protect against a zero row-sum
    row_sum = theta.sum(axis=1, keepdims=True)
    theta   = np.divide(theta, row_sum,
                        out=np.zeros_like(theta), where=row_sum>0)

    spill = pd.DataFrame(theta*100, index=Y.columns, columns=Y.columns)
    TSI = 100 * (theta.sum() - np.trace(theta)) / K
    return spill, TSI


 ## 6 · End‑to‑end driver

In [ ]:

def process_area(area: str):
    clear_output(wait=True)
    display(Markdown(f"## Running pipeline for **{area.capitalize()}** ..."))

    # 1 · prices & returns ------------------------------------------
    prices = load_prices(area)
    show('Intraday prices', prices)

    rets   = simple_returns(prices)
    show('Hourly simple returns', rets)

    # 2 · daily semicov ---------------------------------------------
    pos, neg = daily_semicov(rets)
    show('ReCov⁺ (first day)', pos)
    show('ReCov⁻ (first day)', neg)

    # optional long‑format save
    flat_save(pos, 'pos', area); flat_save(neg, 'neg', area)

    # 3 · vech + PIT ------------------------------------------------
    v_pos, labels = build_vech_dataframe(pos)
    v_neg, _      = build_vech_dataframe(neg)
    show('vech ReCov⁺', v_pos)
    show('vech ReCov⁻', v_neg)

    pit_pos = pit_transform(v_pos)
    pit_neg = pit_transform(v_neg)
    show('PIT ReCov⁺', pit_pos)
    show('PIT ReCov⁻', pit_neg)

    # 4 · write Parquets -------------------------------------------
    path_pos = f"{PARQUET_DIR}/pit_transformed_vech_pos_{area}.parquet"
    path_neg = f"{PARQUET_DIR}/pit_transformed_vech_neg_{area}.parquet"
    pit_pos.to_parquet(path_pos)
    pit_neg.to_parquet(path_neg)
    with open(f"{PARQUET_DIR}/vech_labels_{area}.json", 'w') as f:
        json.dump(labels, f)
    display(Markdown('✅ **Data prep complete – running MHAR spillovers...**'))

    # 5 · MHAR spillovers ------------------------------------------
    spill_p, tsi_p = mhar_spillover(path_pos, f'pos_{area}')
    display(Markdown('### Spillover table – ReCov⁺'))
    display(spill_p.round(2).head())
    display(Markdown(f"**TSI⁺ = {tsi_p:.2f}%**"))

    spill_n, tsi_n = mhar_spillover(path_neg, f'neg_{area}')
    display(Markdown('### Spillover table – ReCov⁻'))
    display(spill_n.round(2).head())
    display(Markdown(f"**TSI⁻ = {tsi_n:.2f}%**"))

    return spill_p, tsi_p, spill_n, tsi_n      



# ------------------------------------------------------------------
# Interactive driver (Jupyter widgets)
# ------------------------------------------------------------------

area_dd = widgets.Dropdown(options=['europe', 'australia'],
                           value='europe', description='Dataset:')
run_btn = widgets.Button(description='Run pipeline', button_style='success')

def _run_pipeline(b):
    clear_output(wait=True)
    display(area_dd, run_btn)
    process_area(area_dd.value)

run_btn.on_click(_run_pipeline)

display(area_dd, run_btn)



## Running pipeline for **Australia** ...

### Intraday prices

Area,nsw,qld,sa,tas,vic
Start DateTime,,,,,
2009-07-01 00:00:00,16.941263,17.650000,16.730280,15.671540,15.50000
2009-07-01 00:05:00,17.709524,18.810089,17.820490,16.057039,15.50000
2009-07-01 00:10:00,17.678644,18.617599,18.123159,15.902460,15.39000
2009-07-01 00:15:00,16.736212,18.611300,17.623659,14.273130,12.81297
2009-07-01 00:20:00,15.638840,17.650000,16.334089,13.241490,11.80000


### Hourly simple returns

Area,nsw,qld,sa,tas,vic,Date
Start DateTime,,,,,,
2009-07-01 00:05:00,0.044350,0.063658,0.063129,0.024301,0.000000,2009-07-01
2009-07-01 00:10:00,-0.001745,-0.010286,0.016842,-0.009674,-0.007122,2009-07-01
2009-07-01 00:15:00,-0.054783,-0.000338,-0.027948,-0.108095,-0.183260,2009-07-01
2009-07-01 00:20:00,-0.067817,-0.053033,-0.075988,-0.075024,-0.082358,2009-07-01
2009-07-01 00:25:00,-0.130264,-0.115418,-0.311559,-0.173825,0.086609,2009-07-01


...

,nsw,nsw-qld,nsw-sa,nsw-tas,nsw-vic,qld,qld-sa,qld-tas,qld-vic,sa,sa-tas,sa-vic,tas,tas-vic,vic
2009-07-01,-0.014553,NaN,-0.358580,NaN,NaN,2.368881,NaN,NaN,NaN,0.872576,NaN,NaN,NaN,NaN,-0.136366
2009-07-02,0.944440,NaN,2.265107,NaN,NaN,2.368881,NaN,NaN,NaN,1.000248,NaN,NaN,NaN,NaN,2.735396
2009-07-03,-0.274959,NaN,-0.439311,NaN,NaN,-0.211336,NaN,NaN,NaN,0.462889,NaN,NaN,NaN,NaN,-0.204807
2009-07-04,-0.385206,NaN,-0.806923,NaN,NaN,-0.607998,NaN,NaN,NaN,1.061850,NaN,NaN,NaN,NaN,-0.564997
2009-07-05,-0.175170,NaN,-0.369959,NaN,NaN,-0.382913,NaN,NaN,NaN,0.436967,NaN,NaN,NaN,NaN,-0.210610


### PIT ReCov⁻

,nsw,nsw-qld,nsw-sa,nsw-tas,nsw-vic,qld,qld-sa,qld-tas,qld-vic,sa,sa-tas,sa-vic,tas,tas-vic,vic
2009-07-01,0.206257,NaN,-0.361610,NaN,NaN,2.368881,NaN,NaN,NaN,1.349156,NaN,NaN,NaN,NaN,0.127057
2009-07-02,0.895662,NaN,2.247017,NaN,NaN,2.368881,NaN,NaN,NaN,0.995576,NaN,NaN,NaN,NaN,2.819190
2009-07-03,-0.339717,NaN,-0.497143,NaN,NaN,-0.270539,NaN,NaN,NaN,0.774893,NaN,NaN,NaN,NaN,-0.309783
2009-07-04,-0.550896,NaN,-1.020352,NaN,NaN,-0.878826,NaN,NaN,NaN,1.123758,NaN,NaN,NaN,NaN,-0.833757
2009-07-05,-0.239005,NaN,-0.403621,NaN,NaN,-0.364643,NaN,NaN,NaN,0.826743,NaN,NaN,NaN,NaN,-0.329207


✅ **Data prep complete – running MHAR spillovers...**

FileNotFoundError: [Errno 2] No such file or directory: 'parquet_files/vech_labels_pos_australia.json'